# Character Consistent Storyboarding with Amazon Nova

This notebook demonstrates how to create character-consistent storyboards using Amazon Nova Canvas and Amazon Nova Reel on Amazon Bedrock. We'll explore prompt engineering techniques to maintain visual consistency across multiple scenes.

## Introduction

Storyboarding is a critical visualization tool in content production across filmmaking, animation, advertising, and UX design. While generative AI models like Amazon Nova Canvas and Nova Reel can generate concepts quickly, maintaining consistent character designs and stylistic coherence across scenes remains challenging.

In this notebook, we'll demonstrate prompt engineering techniques to achieve character consistency without fine-tuning the model.

## Prerequisites
- AWS account with access to Amazon Bedrock
- Appropriate IAM permissions for Bedrock
- The required Python packages (installed in the first cell)

In [ ]:
%pip install -r requirements.txt

## Setup

First, let's import the necessary libraries and helper functions. These include:
- AWS SDK for Python (boto3) for interacting with Amazon Bedrock
- Helper functions for generating text and images
- Display functions for visualizing our storyboards
- Prompt templates for consistent results

In [ ]:
import boto3
import base64
import json
import random

from helpers.bedrock_helpers import generate_text, generate_images, get_random_seed
from helpers.display_helpers import display_story_table, display_prompt_table, display_storyboard
from helpers.prompt_helpers import system_prompts, get_character_descriptions, get_style_prompt

bedrock_client = boto3.client("bedrock-runtime")
bedrock_client_us_east = boto3.client("bedrock-runtime", region_name="us-east-1")

## Story Setup

Now we'll define the basic parameters for our storyboard:
- Number of scenes
- Genre
- Basic story idea
- Characters (initially empty, we'll add them later)

In [ ]:
scene_count = 3
genre = "Family"
idea = "a small robot is lost in a library and needs to read some books to find its way out"
characters = []

## Prompt Creation

We'll create a prompt for the foundation model to generate our story structure. This prompt instructs the model to act as a director and create scenes for our storyboard based on our story idea and genre.

In [ ]:
user_prompt = f"""
        Human: You are an award wining director, create {scene_count} scenes from the <StoryIdea> for storyboarding. The genre is {genre}.
            <StoryIdea>
            {idea}
            </StoryIdea>
        Use the specified characters in the story.  You can add more if needed.
            <Characters>
            {json.dumps(characters)}
            </Characters>
"""

## Story Generation

Now we'll use Amazon Bedrock to generate our story structure. The `generate_text` function will send our prompt to the model and parse the response into a structured format. The `display_story_table` function will then present the story in a readable table format.

In [ ]:
story = generate_text(bedrock_client, user_prompt, system_prompts["story"])
display_story_table(story)

## System Prompt Examination

Let's examine the system prompt used for story generation. This helps us understand how the model is being instructed to structure the story output.

In [ ]:
# Checkout the system prompt (display json)

## Image Prompt Creation

Now we'll create prompts for generating storyboard images. For character consistency, we'll use a specific style and detailed descriptions of each scene. The style parameter can be adjusted to achieve different visual aesthetics.

In [ ]:
style = "3D"

image_prompts = {}
for scene in story.get("scenes"):
    print("Processing scene {}/{} ...".format(scene["scene_id"]+1, story.get("scene_count")))
    base_image_prompt = f"""Generate a storyboard image for the following scene: 
            Scene decription: 
            {scene["description"]}
            Scene imagary: 
            {scene["imagery"]}
            Character descriptions: 
            {get_character_descriptions(scene)}
            """
    conditioned_image_prompt = generate_text(bedrock_client, base_image_prompt, system_prompts["image"])
    styled_image_prompt = generate_text(bedrock_client, f"Image generation prompt: {conditioned_image_prompt}", get_style_prompt(style))
    image_prompts[scene["scene_id"]] = styled_image_prompt["prompt"]
display_prompt_table(story_data=story, image_prompts=image_prompts)

## Image Generation & Display Storyboard


Now we'll generate storyboard images using Amazon Nova Canvas. For each scene, we'll generate multiple images with the same seed to maintain consistency. The negative prompt helps avoid unwanted elements in the generated images.

Finally, let's display our complete storyboard with the generated images. This will show each scene with its description and the corresponding images. You can select different images for each scene to create your final storyboard.

In [ ]:
images_per_scene = 3
negative_prompt = "ugly, yellow, red, green, blue, pink, orange, brown, old, dirty, colorful"
seed = get_random_seed()

storyboard_images = {}
for scene in story.get("scenes"):
    scene_id = scene["scene_id"]
    print("Processing scene {}/{} ...".format(scene_id+1, story.get("scene_count")))
    prompt = image_prompts[scene_id]
    images = generate_images(bedrock_client_us_east, prompt, negative_prompt, seed=seed, image_count=images_per_scene)
    storyboard_images[scene_id] = images

display_storyboard(storyboard_images, story)

## Conclusion

In this notebook, we've demonstrated how to create character-consistent storyboards using Amazon Nova Canvas through careful prompt engineering. Key techniques include:

1. Detailed scene descriptions with specific character attributes
2. Consistent style prompts across all scenes
3. Using the same seed value for all image generations
4. Negative prompts to avoid unwanted elements

For even greater character consistency, consider exploring the fine-tuning approach demonstrated in Part 2 of this series.